In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import

In [ ]:
import sys
from logging import INFO, StreamHandler, getLogger

logger = getLogger()
if not logger.hasHandlers():
    logger.addHandler(StreamHandler(sys.stdout))
logger.setLevel(INFO)

In [ ]:
import os
import pathlib

import torch
from src.four_dim_srda.config.experiment_config import (
    CFDConfig,
    CFDTimeConfig,
    DAConfig,
    SeedConfig,
)
from src.qg_model.utils.config import EslerJetConfig

# Define constant

In [ ]:
ROOT_DIR = pathlib.Path(os.environ["PYTHONPATH"]).parent.resolve()

experiment_name = "experiment7"
config_dir_path = (
    f"{ROOT_DIR}/python/configs/four_dim_srda/{experiment_name}/cfd_simulation/qg_model"
)
os.makedirs(config_dir_path, exist_ok=True)

# Make & Save config

In [ ]:
output_lr_dt = 1.0
output_scale_factor = 4
lr_dt = 1e-2

time_config = CFDTimeConfig(
    start_time=0,
    end_time=200,
    lr_dt=lr_dt,
    hr_dt=lr_dt / 4,
    uhr_dt=lr_dt / 8,
    output_lr_dt=output_lr_dt,
    output_hr_dt=output_lr_dt / output_scale_factor,
    output_uhr_dt=output_lr_dt / output_scale_factor,
)

seed_config = SeedConfig(
    seed_start=0,
    seed_end=10,
    uhr_seed_start=9990,
    uhr_seed_end=9999,
)

In [ ]:
assimilation_interval = int(time_config.output_lr_dt / time_config.output_hr_dt)
forecast_span = 4

data_assimilation_config = DAConfig(
    assimilation_dt=output_lr_dt,
    assimilation_interval=assimilation_interval,
    forecast_span=forecast_span,
    segment_length=2 * forecast_span + 1,
)

In [ ]:
hr_nx, hr_ny, hr_nz = 128, 65, 16
lr_nx, lr_ny, lr_nz = 32, 17, 4
uhr_nx, uhr_ny, uhr_nz = 256, 129, 32


def calculate_cutoff_wavenumbers(nx, ny):
    #
    ny = ny - 1

    # Calculate the cutoff wavenumber for kx
    kx_cutoff = nx // 3

    # Calculate reflected_ny for ky calculation
    reflected_ny = (ny - 1) * 2

    # Calculate the cutoff wavenumber for ky
    ky_cutoff = reflected_ny // 3

    logger.info(f"Cutoff wavenumber for kx: {kx_cutoff}")
    logger.info(f"Cutoff wavenumber for ky: {ky_cutoff}")

    return min(kx_cutoff, ky_cutoff)


cutoff_wavenum = calculate_cutoff_wavenumbers(hr_nx, hr_ny)

In [ ]:
# Characteristic scale, referring to Esler(2008)
characteristic_length = 800e3
characteristic_velocity = 40
characteristic_time = characteristic_length / characteristic_velocity

# Channel
hx, hy, hz = 2 * torch.pi, torch.pi, 10e3 / characteristic_length

# Coriolis frequency
f_0 = 1e-4 * characteristic_time

# Beta parameter, referring to Esler(2008)
beta = 1.6e-11 * characteristic_length**2 / characteristic_velocity

# Reduced gravity
brunt_vaisala_frequency = 0.01 * characteristic_time  # referring to Valis (2017) p99
dH_hr = hz / hr_nz
dH_lr = hz / lr_nz
dH_uhr = hz / uhr_nz
reduced_gravity_hr = brunt_vaisala_frequency**2 * dH_hr
reduced_gravity_lr = brunt_vaisala_frequency**2 * dH_lr
reduced_gravity_uhr = brunt_vaisala_frequency**2 * dH_uhr

# Jet width, referring to Esler(2008)
jet_width = 0.4

# Diffusion exponent
diffusion_exponent = 2

# Diffusion coefficient, referring to Scott & Polvani(2007)
diffusion_coefficient = 30 / cutoff_wavenum ** (2 * diffusion_exponent)

In [ ]:
width_z = 0.125 * hz

In [ ]:
n_batch = 20

In [ ]:
# device= gpu

config_path = f"{config_dir_path}/gpu_make_data_config.yml"

hr_base_config = EslerJetConfig(
    nx=hr_nx,
    ny=hr_ny,
    nz=hr_nz,
    n_batch=n_batch,
    f_0=f_0,
    beta=beta,
    Lx=hx,
    Ly=hy,
    Lz=hz,
    reduced_gravity=reduced_gravity_hr,
    diffusion_coefficient=diffusion_coefficient,
    diffusion_exponent=diffusion_exponent,
    device="cuda",
    precision="double",
    jet_width=jet_width,
    jet_max_velocity=1.0,
    noise_amplitude=0.03,
    width_z=width_z,
)

lr_base_config = EslerJetConfig(
    nx=lr_nx,
    ny=lr_ny,
    nz=lr_nz,
    n_batch=n_batch,
    f_0=f_0,
    beta=beta,
    Lx=hx,
    Ly=hy,
    Lz=hz,
    reduced_gravity=reduced_gravity_lr,
    diffusion_coefficient=diffusion_coefficient,
    diffusion_exponent=diffusion_exponent,
    device="cuda",
    precision="double",
    jet_width=jet_width,
    jet_max_velocity=1.0,
    noise_amplitude=0.03,
    width_z=width_z,
)

uhr_base_config = EslerJetConfig(
    nx=uhr_nx,
    ny=uhr_ny,
    nz=uhr_nz,
    n_batch=1,
    f_0=f_0,
    beta=beta,
    Lx=hx,
    Ly=hy,
    Lz=hz,
    reduced_gravity=reduced_gravity_uhr,
    diffusion_coefficient=diffusion_coefficient,
    diffusion_exponent=diffusion_exponent,
    device="cuda",
    precision="double",
    jet_width=jet_width,
    jet_max_velocity=1.0,
    noise_amplitude=0.03,
    width_z=width_z,
)

config = CFDConfig(
    jet_profile="sech_squared_and_sigmoid",
    hr_base_config=hr_base_config,
    lr_base_config=lr_base_config,
    uhr_base_config=uhr_base_config,
    time_config=time_config,
    seed_config=seed_config,
    da_config=data_assimilation_config,
)


config.save(pathlib.Path(config_path))

In [ ]:
# For evaluation
config.lr_base_config.n_batch = 1
config.hr_base_config.n_batch = 1

config_path = f"{config_dir_path}/gpu_evaluation_config.yml"

config.save(pathlib.Path(config_path))